In [ ]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime, Float, select, Date, JSON

import os
import sys

from wikibaseintegrator import wbi_core, wbi_login, wbi_login
from wikibaseintegrator.wbi_config import config as wbi_config

In [ ]:
BRICKIT_BD_LOGIN = 'postgres'
BRICKIT_BD_PASSWORD = 'pass'
BRICKIT_BD_HOST = 'host'
WIKIBASE_HOST = 'wiki_host'

WIKIBASE_LOGIN = 'WikibaseAdmin'
WIKIBASE_PASSWORD = 'WikibaseDockerAdminPass'

connection_string = f'postgresql://{BRICKIT_BD_LOGIN}:{BRICKIT_BD_PASSWORD}@{BRICKIT_BD_HOST}:5432/holybricks'
engine_pg = create_engine(connection_string)

wbi_config['MEDIAWIKI_API_URL'] = f'http://{WIKIBASE_HOST}:8181/api.php'
wbi_config['SPARQL_ENDPOINT_URL'] = f'http://{WIKIBASE_HOST}:8989/bigdata/sparql'
wbi_config['WIKIBASE_URL'] = 'http://wikibase.svc'

In [ ]:
# Пока не удалось найти нормальный способ однозначно получать entity_id сущностей по их алиасам. 
# Допускаем, что это как-то можно и у нас есть такие данные
ITEMS_DICT = {
    'I': {
        'Brickit Company': 'Q1',
        'Brickit Image': 'Q2',
        'Brickit Part': 'Q3'

    },
    'P': {
        'instance of': 'P1',
        'Image URL': 'P2',
        'Image ID': 'P3',
        'Part Name': 'P4',
        'Part Tag': 'P5',
        'Part Image': 'P6',
        'Part Num': 'P7',
        'Part Child': 'P8'
    }    
}

# Забрать какие-то данные из Brickit

In [ ]:
df_images = pd.read_sql(f"""
SELECT id, public_url
FROM staging.manual_images
WHERE initial_entity_type = 'part'
    """, con = engine_pg)


df_parts = pd.read_sql(f"""
SELECT part_num, "name", tag, part_cat_id, child_part_nums, image_id
FROM staging.synthetic_parts
    """, con = engine_pg)

# Залить сущности

In [ ]:
# Костыль для связки наша сущность - ID в Wikibase/
# Предполагаем, что это можно взять при помощи SPARQL
# А пока что по результатам циклов ниже в эти датафреймы дописывается ID Wikibase и сохраняются в csv
# df_images = pd.read_csv('./df_images.csv')
# df_parts = pd.read_csv('./df_parts.csv')

In [ ]:
login_instance = wbi_login.Login(user=WIKIBASE_LOGIN, pwd=WIKIBASE_PASSWORD)   

In [ ]:
# Загрузить 100 изображений из всех
for _, img_i in df_images[:100].iterrows():
    data = [
        wbi_core.Url(str(img_i['public_url']), prop_nr=ITEMS_DICT['P']['Image URL']),
        wbi_core.String(str(img_i['id']), prop_nr=ITEMS_DICT['P']['Image ID']),
        wbi_core.ItemID(ITEMS_DICT['I']['Brickit Image'], prop_nr=ITEMS_DICT['P']['instance of'])
    ]
    item = wbi_core.ItemEngine(new_item=True, data=data,core_props=set())
    
    # Этот метод в библиотеке из коробки не работает. 
    # Надо либо закомментить в библиотеке в wbi_core.ItemEngine.set_label() условие после "Skip set_label if the item already have one and if_exists is at 'KEEP'"
    # Либо просто не проставлять лейблы
    item.set_label('img_' + str(img_i['id']), if_exists='REPLACE')
    
    r = item.write(login_instance)    
    
    df_images.loc[df_images.id == img_i['id'], 'entity_id'] = r

In [ ]:
df_images.to_csv('./df_images.csv', index = False) #Обновляем "базу знаний" про связки с id Wikibase

In [ ]:
# Заливка деталей
df_parts_img = df_parts[df_parts.image_id.isin(df_images[~df_images.entity_id.isnull()].id)] #только тех, для которых мы уже залили картинки
for _, part_i in df_parts_img.iterrows():
    part_img = part_i['image_id']
    img_Q = df_images[df_images.id == part_img].reset_index().at[0, 'entity_id']
    
    data = [
        wbi_core.String(str(part_i['name']), prop_nr=ITEMS_DICT['P']['Part Name']),
        wbi_core.String(str(part_i['tag']), prop_nr=ITEMS_DICT['P']['Part Tag']),
        wbi_core.String(str(part_i['part_num']), prop_nr=ITEMS_DICT['P']['Part Num']),
        wbi_core.ItemID(ITEMS_DICT['I']['Brickit Part'], prop_nr=ITEMS_DICT['P']['instance of']),
        wbi_core.ItemID(img_Q, prop_nr=ITEMS_DICT['P']['Part Image'])
    ]
    item = wbi_core.ItemEngine(new_item=True, data=data,core_props=set())
    
    # Этот метод в библиотеке из коробки не работает. 
    item.set_label('part_num_' + str(part_i['part_num']), if_exists='REPLACE')
    
    r = item.write(login_instance)    
    
    df_parts.loc[df_parts.part_num == part_i['part_num'], 'entity_id'] = r

In [ ]:
df_parts.to_csv('./df_parts.csv', index = False) #Обновляем "базу знаний" про связки с id Wikibase

# Прочие знания

In [ ]:
# API поиска
wbi_core.ItemEngine.get_search_results('part_num_10793') 

In [ ]:
# SPARQL
wbi_core.ItemEngine.execute_sparql_query("""""") 

In [ ]:
# Дескрипшн
set_description(self, description, lang=None, if_exists='REPLACE'):

In [ ]:
# Обновление существующих айтемов
data = [
    wbi_core.ItemID(img_Q, prop_nr = ITEMS_DICT['P']['Part Image'])
]
item = wbi_core.ItemEngine(new_item=False, item_id = 'Q1234', data=data,core_props=set())
r = item.write(login_instance)  